In [14]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [30]:
class ResidualBlock(nn.Module):
  def __init__(self,in_channels,out_channels,kernel_size,stride = 1):
    super(ResidualBlock,self).__init__()
    self.conv1 = nn.Conv2d(in_channels=in_channels,out_channels=out_channels,kernel_size = (1,1),stride = 1)
    self.bn1 = nn.BatchNorm2d(out_channels)
    self.relu1 = nn.ReLU()
    self.conv2 = nn.Conv2d(in_channels=out_channels, out_channels=out_channels,kernel_size = (3,3),stride = 1)
    self.bn2 = nn.BatchNorm2d(out_channels)
    self.relu2 = nn.ReLU()
    self.conv3 = nn.Conv2d(in_channels=out_channels,out_channels=out_channels*4,kernel_size=(1,1),stride = 1)
    self.bn3 = nn.BatchNorm2d(out_channels)
    self.relu3 = nn.ReLU()
    self.shortcut = nn.Sequential()
    if stride != 1 or in_channels != out_channels*4:
      self.shortcut = nn.Sequential(
          nn.Conv2d(in_channels = in_channels,out_channels=out_channels*4,kernel_size = (1,1),stride = stride,padding = 0),
          nn.BatchNorm2d(out_channels*4)
      )

  def forward(self,x):
    output = self.conv1(x)
    output = self.bn1(output)
    output = self.relu1(output)
    output = self.conv2(output)
    output = self.bn2(output)
    output = self.relu2(output)
    output = self.conv3(output)
    output = self.bn3(output)
    output = self.relu3(output)
    self.shortcut = self.shortcut(x)
    output += self.shortcut
    output = self.relu3(output)
    return output




In [36]:
class myResnet50(nn.Module):
  def __init__(self,num_classes = 1000):
    super(myResnet50,self).__init__()
    #First layer
    self.conv1 = nn.Conv2d(in_channels=64,out_channels=64,kernel_size = (7,7),stride = 2)
    self.bn1 = nn.BatchNorm2d(64)
    self.relu1 = nn.ReLU()
    self.maxpool1 = nn.MaxPool2d(kernel_size = (3,3),stride = 2)

    #Residual blocks
    self.stage1 = self._make_stage(64, 64, stride=1, num_blocks=3)
    self.stage2 = self._make_stage(256, 128, stride=2, num_blocks=4)
    self.stage3 = self._make_stage(512, 256, stride=2, num_blocks=6)
    self.stage4 = self._make_stage(1024, 512, stride=2, num_blocks=3)

    #fully connected
    self.avgpool = nn.AdaptiveAvgPool2d((1,1))
    self.fc = nn.Linear(2048,num_classes)

  def _make_stage(self,in_channels,out_channels,stride,num_blocks):
    layers = []
    layers.append(ResidualBlock(in_channels,out_channels,stride))
    for _ in range(1,num_blocks):
      layers.append(ResidualBlock(out_channels*4,out_channels,1))
    return nn.Sequential(*layers)

  def forward(self,x):
    #First layer
    output = self.conv1(x)
    output = self.bn1(output)
    output = self.relu1(output)
    output = self.maxpool1(output)

    #Stages
    output = self.stage1(x)
    output = self.stage2(x)
    output = self.stage3(x)
    output = self.stage4(x)

    #Fully connected
    output = self.avgpool(output)
    output = output.view(output.size(0),-1) #Flatten the layer
    output = self.fc(output)
    return output



In [37]:
model = myResnet50(num_classes = 1000)
print(model)

myResnet50(
  (conv1): Conv2d(64, 64, kernel_size=(7, 7), stride=(2, 2))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (maxpool1): MaxPool2d(kernel_size=(3, 3), stride=2, padding=0, dilation=1, ceil_mode=False)
  (stage1): Sequential(
    (0): ResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU()
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu2): ReLU()
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
      (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu3): ReLU()
      (shortcut): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
        (1): BatchNorm2d(256, eps=1e-05, mo